In [15]:
import pandas as pd
import numpy as np
import pulp

# Put in place all data

In [16]:
master = pd.read_csv('pokemon_master_data.csv', encoding='latin-1')
del master['Unnamed: 0']

In [17]:
damaging_moves = pd.read_csv('damaging_moves.csv')
del damaging_moves['Unnamed: 0']

In [18]:
damaging_moves['Type'] = damaging_moves['Type'].apply(lambda x: x.capitalize())
damaging_moves['Expected Damage'] = damaging_moves['Damage'] * damaging_moves['Acc'] / 100

In [19]:
with open ('moves_matrix.csv') as f:
    moves_matrix = f.read()
moves_matrix = [int(m) for m in moves_matrix.split(',')]
moves_matrix = [moves_matrix[x:x+len(master)] for x in range(0, len(moves_matrix), len(damaging_moves))]

In [20]:
types_order = ['Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fighting', 'Poison', 'Ground', 'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy']

In [21]:
for t in types_order:
    damaging_moves[t] = (damaging_moves['Type'] == t).astype('int')

In [22]:
weaknesses = {
    'Normal': [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    'Fire': [1, 0.5, 2, 1, 0.5, 0.5, 1, 1, 2, 1, 1, 0.5, 2, 1, 1, 1, 0.5, 0.5],
    'Water': [1, 0.5, 0.5, 2, 2, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 1],
    'Electric': [1, 1, 1, 0.5, 1, 1, 1, 1, 2, 0.5, 1, 1, 1, 1, 1, 1, 0.5, 1],
    'Grass': [1, 2, 0.5, 0.5, 0.5, 2, 1, 2, 0.5, 2, 1, 2, 1, 1, 1, 1, 1, 1],
    'Ice': [1, 2, 1, 1, 1, 0.5, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1],
    'Fighting': [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0.5, 0.5, 1, 1, 0.5, 1, 2],
    'Poison': [1, 1, 1, 1, 0.5, 1, 0.5, 0.5, 2, 1, 2, 0.5, 1, 1, 1, 1, 1, 0.5],
    'Ground': [1, 1, 2, 0, 2, 2, 1, 0.5, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 1],
    'Flying': [1, 1, 1, 2, 0.5, 2, 0.5, 1, 0, 1, 1, 0.5, 2, 1, 1, 1, 1, 1],
    'Psychic': [1, 1, 1, 1, 1, 1, 0.5, 1, 1, 1, 0.5, 2, 1, 2, 1, 2, 1, 1],
    'Bug': [1, 2, 1, 1, 0.5, 1, 0.5, 1, 0.5, 2, 1, 1, 2, 1, 1, 1, 1, 1],
    'Rock': [0.5, 0.5, 2, 1, 2, 1, 2, 0.5, 2, 0.5, 1, 1, 1, 1, 1, 1, 2, 1],
    'Ghost': [0, 1, 1, 1, 1, 1, 0, 0.5, 1, 1, 1, 0.5, 1, 2, 1, 2, 1, 1],
    'Dragon': [1, 0.5, 0.5, 0.5, 0.5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2],
    'Dark': [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 0.5, 1, 0.5, 1, 2],
    'Steel': [0.5, 2, 1, 1, 0.5, 0.5, 2, 0, 2, 0.5, 0.5, 0.5, 0.5, 1, 0.5, 1, 0.5, 0.5],
    'Fairy': [1, 1, 1, 1, 1, 1, 0.5, 2, 1, 1, 1, 0.5, 1, 1, 0, 0.5, 2, 1]
}

In [23]:
w_arrays = []
r_arrays = []
for pkmn in range(len(master)):
    if str(master.loc[pkmn]['Type2']) != 'nan':
        w_array_1 = np.array(weaknesses[master.loc[pkmn]['Type1']])
        w_array_2 = np.array(weaknesses[master.loc[pkmn]['Type2']])
        w_array = w_array_1 * w_array_2
    else:
        w_array = np.array(weaknesses[master.loc[pkmn]['Type1']])
    r_array = [1 if r <= 0.5 else 0 for r in w_array]
    w_arrays.append(w_array)
    r_arrays.append(r_array)
master['Weaknesses'] = w_arrays
master['Resistances'] = r_arrays

In [24]:
starters_n = ['Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle', 'Blastoise', 'Chikorita', 'Bayleef', 'Meganium', 'Cyndaquil', 'Quilava', 'Typhlosion', 'Totodile', 'Croconaw', 'Feraligatr', 'Treecko', 'Grovyle', 'Sceptile', 'Torchic', 'Combusken', 'Blaziken', 'Mudkip', 'Marshtomp', 'Swampert', 'Turtwig', 'Grotle', 'Torterra', 'Chimchar', 'Monferno', 'Infernape', 'Piplup', 'Prinplup', 'Empoleon', 'Snivy', 'Servine', 'Serperior', 'Tepig', 'Pignite', 'Emboar', 'Oshawott', 'Dewott', 'Samurott', 'Chespin', 'Quilladin', 'Chesnaught', 'Fennekin', 'Braixen', 'Delphox', 'Froakie', 'Frogadier', 'Greninja', 'Rowlet', 'Dartrix', 'Decidueye', 'Litten', 'Torracat', 'Incineroar', 'Popplio', 'Brionne', 'Primarina']
pseudolegendaries_n = ['Dratini', 'Dragonair', 'Dragonite', 'Larvitar', 'Pupitar', 'Tyranitar', 'Bagon', 'Shelgon', 'Salamence', 'Beldum', 'Metang', 'Metagross', 'Gible', 'Gabite', 'Garchomp', 'Deino', 'Zweilous', 'Hydreigon', 'Goomy', 'Sliggoo', 'Goodra', 'Jangmo-o', 'Hakamo-o', 'Kommo-o']
legendaries_n = ['Articuno', 'Zapdos', 'Moltres', 'Mewtwo', 'Mew', 'Raikou', 'Entei', 'Suicune', 'Lugia', 'Celebi', 'Regirock', 'Regice', 'Registeel', 'Latias', 'Latios', 'Kyogre', 'Groudon', 'Rayquaza', 'Jirachi', 'Deoxys', 'Uxie', 'Mesprit', 'Azelf', 'Dialga', 'Palkia', 'Giratina', 'Cresselia', 'Darkrai', 'Manaphy', 'Heatran', 'Regigigas', 'Arceus', 'Victini', 'Cobalion', 'Terrakion', 'Virizion', 'Keldeo', 'Tornadus', 'Thundurus', 'Landorus', 'Reshiram', 'Zekrom', 'Kyurem', 'Genesect', 'Xerneas', 'Yveltal', 'Zygarde', 'Diancie', 'Volcanion', 'Solgaleo', 'Lunala', 'Marshadow', 'Zeraora', 'Cosmog', 'Cosmoem', 'Necrozma', 'Silvally', 'Magearna']
ultrabeasts_n = ['Nihilego', 'Buzzwole', 'Pheromosa', 'Xurkitree', 'Celesteela', 'Kartana', 'Guzzlord', 'Poipole', 'Naganadel', 'Stakataka', 'Blacephalon']

In [25]:
starters = [master[master['Name']==name].index[0] for name in starters_n]
pseudolegendaries = [master[master['Name']==name].index[0] for name in pseudolegendaries_n]
legendaries = [master[master['Name']==name].index[0] for name in legendaries_n]
ultrabeasts = [master[master['Name']==name].index[0] for name in ultrabeasts_n]

In [26]:
master['TotalStats'] = master['Attack'] + master['Special Attack'] + master['Defence'] + master['Special Defence'] + master['Speed'] + master['HP']
master['TotalAttack'] = master['Attack'] + master['Special Attack']
master['TotalDefence'] = master['Defence'] + master['Special Defence'] + master['HP']

# The linear program

In [27]:
def solve_linear_program(objective_stat='TotalStats', max_pkmn_num=792, num_starters=1, num_pseudos=0, num_legendaries=0, num_ultrabeasts=0):
    # Solve the first linear program (choose pokemon team)
    prob1 = pulp.LpProblem("PerfectPokemonTeam", pulp.LpMaximize)
    x = pulp.LpVariable.dicts("x", range(max_pkmn_num), cat=pulp.LpBinary)
    
    objective_function1 = sum(master.loc[pkmn, objective_stat] * x[pkmn] for pkmn in range(max_pkmn_num))
    prob1 += objective_function1
    
    prob1 += sum([x[pkmn] for pkmn in range(max_pkmn_num)]) == 6
    prob1 += sum([x[pkmn] for pkmn in [i for i in starters if i < max_pkmn_num]]) <= num_starters
    prob1 += sum([x[pkmn] for pkmn in [i for i in pseudolegendaries if i < max_pkmn_num]]) == num_pseudos
    prob1 += sum([x[pkmn] for pkmn in [i for i in legendaries if i < max_pkmn_num]]) == num_legendaries
    prob1 += sum([x[pkmn] for pkmn in [i for i in ultrabeasts if i < max_pkmn_num]]) == num_ultrabeasts
        
    for tp in range(18):
        prob1 += sum([master.loc[pkmn, 'Resistances'][tp] * x[pkmn] for pkmn in range(max_pkmn_num)]) >= 1
        
    prob1.solve()
    team = [i for i in range(max_pkmn_num) if x[i].value() == 1]
    
    
    # Solve the second linear program (choose the moves)
    prob2 = pulp.LpProblem("PerfectPokemonTeam", pulp.LpMaximize)
    y = pulp.LpVariable.dicts("y", ((i, j) for i in team for j in range(len(damaging_moves))), cat=pulp.LpBinary)

    objective_function2 = sum(damaging_moves.loc[mv, 'Expected Damage'] * y[pkmn, mv] for mv in range(len(damaging_moves)) for pkmn in team) + sum(0.5 * int(damaging_moves.loc[mv, 'Type'] in set(master.loc[pkmn, ['Type1', 'Type2']])) * damaging_moves.loc[mv, 'Expected Damage'] * y[pkmn, mv] for mv in range(len(damaging_moves)) for pkmn in team)
    prob2 += objective_function2
    
    for pkmn in team:
        prob2 += sum([y[pkmn, mv] for mv in range(len(damaging_moves))]) == 4
        prob2 += y[pkmn, 102] + y[pkmn, 294] <= 0  # Explude self-destruct and explosion
    
    for pkmn in team:
        for mv in range(len(damaging_moves)):
            prob2 += y[pkmn, mv] <= moves_matrix[pkmn][mv]
    
    for tp in types_order:
        prob2 += sum([y[pkmn, mv]*damaging_moves.loc[mv, tp] for mv in range(len(damaging_moves)) for pkmn in team]) >= 1
    
    prob2.solve()

    moves = [(i, [m for m in range(len(damaging_moves)) if y[i, m].value() == 1]) for i in team]
    team_moves = {master.loc[moves[i][0], 'Name']: [damaging_moves.loc[moves[i][1][j], 'Move name'] for j in range(4)] for i in range(6)}
    return team_moves

In [28]:
solve_linear_program()

{'Arcanine': ['BurnUp', 'Crunch', 'FlareBlitz', 'Overheat'],
 'Archeops': ['Earthquake', 'HeadSmash', 'ShadowClaw', 'SkyAttack'],
 'Florges': ['GigaImpact', 'Moonblast', 'Psychic', 'SolarBeam'],
 'Haxorus': ['DracoMeteor', 'Outrage', 'PoisonJab', 'Surf'],
 'Magnezone': ['IronHead', 'SignalBeam', 'Thunderbolt', 'WildCharge'],
 'Slaking': ['FocusPunch', 'GigaImpact', 'HyperBeam', 'IceBeam']}